In [ ]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1579220448498_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
import time

In [ ]:
start = time.time()

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

import datetime
from pyspark.sql.functions import *
from pyspark.sql.functions import year, month, dayofmonth,dayofweek
from pyspark.sql.window import Window

ss = SparkSession.builder.getOrCreate()

In [ ]:
aqi_df = ss.read.format("csv").option("header", "true").load(
                "s3a://dc-group2/aqi_year_countycode/combined_aqi.csv")

In [ ]:
temp_df = ss.read.format("csv").option("header", "true").load(
                "s3a://dc-group2/aqi_year_countycode/combined_csv_temp.csv")

In [ ]:
humidity_df = ss.read.format("csv").option("header", "true").load(
    "s3a://dc-group2/aqi_year_countycode/combined_csv_RH&Dewpoint.csv")

In [ ]:
wind_df = ss.read.format("csv").option("header", "true").load(
                "s3a://dc-group2/aqi_year_countycode/combined_csv_wind.csv")

In [ ]:
pressure_df = ss.read.format("csv").option("header", "true").load(
                "s3a://dc-group2/aqi_year_countycode/combined_csv_pressure.csv")

In [ ]:
aqi_df = aqi_df.withColumn("AQI_num", aqi_df['AQI'].astype('float'))

In [ ]:
aqi_df = aqi_df.drop("Defining Site", "Number of Si.tes Reporting",
                    "Defining Parameter", "Category")\
                .groupBy(["State Name", "County Name", \
                            "State Code", "County Code","Date Local"])\
                        .avg()

In [ ]:
temp_df = temp_df.withColumn("temp", temp_df["Arithmetic Mean"].astype("float"))

In [ ]:
temp_df = temp_df.groupBy(["State Name", "County Name", \
                            "State Code", "County Code","Date Local"])\
                        .avg("temp")

In [ ]:
pressure_df = pressure_df.withColumn("pressure", pressure_df["Arithmetic Mean"].astype("float"))

In [ ]:
pressure_df = pressure_df.groupBy(["State Name", "County Name", \
                            "State Code", "County Code","Date Local"])\
                        .avg("pressure")

In [ ]:
humidity_df = humidity_df.withColumn("humidity", humidity_df["Arithmetic Mean"].astype("float"))

In [ ]:
humidity_df = humidity_df.withColumnRenamed("Parameter Name", "Parameter_Name")\
                        .where("Parameter_Name == 'Relative Humidity '")

In [ ]:
humidity_df = humidity_df.groupBy(["State Name", "County Name", \
                            "State Code", "County Code","Date Local"])\
                        .avg("humidity")

In [ ]:
wind_speed_df = wind_df.withColumn("wind_speed", wind_df["Arithmetic Mean"].astype("float"))\
                .withColumnRenamed("Parameter Name", "Parameter_Name")\
                        .where("Parameter_Name == 'Wind Speed - Resultant'")

In [ ]:
wind_speed_df = wind_speed_df.groupBy(["State Name", "County Name", \
                            "State Code", "County Code","Date Local"])\
                        .avg("wind_speed")

In [ ]:
wind_direction_df = wind_df.withColumn("wind_direction", wind_df["Arithmetic Mean"].astype("float"))\
                .withColumnRenamed("Parameter Name", "Parameter_Name")\
                        .where("Parameter_Name == 'Wind Direction - Resultant'")

In [ ]:
wind_direction_df = wind_direction_df.groupBy(["State Name", "County Name", \
                            "State Code", "County Code","Date Local"])\
                        .avg("wind_direction")

In [ ]:
join1 = aqi_df.join(temp_df, ["State Name", "County Name", "State Code", "County Code","Date Local"], "left_outer")

In [ ]:
join2 = join1.join(humidity_df, ["State Name", "County Name", "State Code", "County Code","Date Local"], "left_outer")

In [ ]:
join3 = join2.join(pressure_df, ["State Name", "County Name", "State Code", "County Code","Date Local"], "left_outer")

In [ ]:
join4 = join3.join(wind_speed_df, ["State Name", "County Name", "State Code", "County Code","Date Local"],
                    "left_outer")

In [ ]:
join_final = join4.join(wind_direction_df, ["State Name", "County Name", "State Code", "County Code","Date Local"],
                    "left_outer")

In [ ]:
join_final.count()

In [ ]:
original_name = ['State Name', 'County Name', 'State Code'
                 , 'County Code', 'Date Local', 'avg(AQI_num)'
                 , 'avg(temp)', 'avg(humidity)', 'avg(pressure)'
                 , 'avg(wind_speed)', 'avg(wind_direction)']
new_name = ['State_Name', 'County_Name', 'State_Code'
            , 'County_Code', 'Date_Local', 'AQI'
            , 'temp', 'humidity', 'pressure'
            , 'wind_speed', 'wind_direction']
for i in range(len(original_name)):
    join_final = join_final.withColumnRenamed(original_name[i],new_name[i])

In [ ]:
# join_final.write\
#     .option("path", "s3a://dc-group2/leona_left")\
#     .saveAsTable("leona_left")
data_left = ss.read.parquet("s3a://dc-group2/leona_left")
data_left = data_left.withColumn("Date", data_left['Date_Local'].astype('Date'))
data_left = data_left.withColumn("State_Code_num"
                                 , data_left['State_Code'].astype('integer'))
data_left = data_left.withColumn("County_Code_num"
                                 , data_left['County_Code'].astype('integer'))

data_left = data_left.select('State_Name',
                               'County_Name',
                               'State_Code_num',
                               'County_Code_num',
                               'Date',
                                year("Date").alias('year'),
                               month("Date").alias('month'),
                               dayofmonth("Date").alias('day'),
                               dayofweek('Date').alias('dow'),
                               'temp',
                               'humidity',
                               'pressure',
                               'wind_speed',
                               'wind_direction',
                               'AQI'
                              )

In [ ]:
LA_left = data_left.where("State_Code_num == 6 and County_Code_num == 37" )

imputer  = feature.Imputer(inputCols = ['temp', 'humidity', 'pressure'
                                        , 'wind_speed', 'wind_direction']
                           ,outputCols=['imputed_temp', 'imputed_humidity'
                                        , 'imputed_pressure', 'imputed_wind_speed'
                                        , 'imputed_wind_direction'])

LA_left_imputed = imputer.setStrategy('median').fit(LA_left).transform(LA_left)
LA_left_imputed = LA_left_imputed.drop('temp', 'humidity', 'pressure'
                                        , 'wind_speed', 'wind_direction')
LA_left_imputed = LA_left_imputed.orderBy('Date')
LA_left_imputed = LA_left_imputed.withColumnRenamed('AQI','label')

In [ ]:
def add_lag(df, n):
    for i in range(1,n+1):
        df = df.withColumn('prev'+str(i), lag("label", i).over(Window.partitionBy(
                                    'State_Code_num','County_Code_num')\
                                                     .orderBy('Date')))
    return df
LA_left_lag30 = add_lag(LA_left_imputed,30)

In [ ]:
end = time.time()
gap = start-end
print(gap)